In [1]:
import sys
import asyncio
import time
import os

import numpy as np

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS

In [2]:
import logging
stream_handler = logging.StreamHandler(sys.stdout)
logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [3]:
print(os.environ["OSPL_URI"])
print(os.environ["LSST_DDS_PARTITION_PREFIX"])

file:///home/patrickingraham/repos/ts_ddsconfig/config/ospl-sp.xml
summit


In [4]:
#get classes and start them
domain = salobj.Domain()
await asyncio.sleep(10) # This can be removed in the future...
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

atmcs: Adding all resources.
atptg: Adding all resources.
ataos: Adding all resources.
atpneumatics: Adding all resources.
athexapod: Adding all resources.
atdome: Adding all resources.
atdometrajectory: Adding all resources.
atcamera: Adding all resources.
atspectrograph: Adding all resources.
atheaderservice: Adding all resources.
atarchiver: Adding all resources.
Read historical data in 0.01 sec
Read 6 history items for RemoteEvent(ATPneumatics, 0, appliedSettingsMatchStart)
Read 12 history items for RemoteEvent(ATPneumatics, 0, cellVentsState)
Read 2 history items for RemoteEvent(ATPneumatics, 0, eStop)
Read 100 history items for RemoteEvent(ATPneumatics, 0, heartbeat)
Read 9 history items for RemoteEvent(ATPneumatics, 0, instrumentState)
Read 38 history items for RemoteEvent(ATPneumatics, 0, m1CoverLimitSwitches)
Read 21 history items for RemoteEvent(ATPneumatics, 0, m1CoverState)
Read 100 history items for RemoteEvent(ATPneumatics, 0, m1SetPressure)
Read 26 history items for Remo

[[None, None, None, None, None, None, None], [None, None, None, None]]

torqueDemand DDS read queue is filling: 25 of 100 elements
mount_positions DDS read queue is filling: 20 of 100 elements
nasymth_m3_mountMotorEncoders DDS read queue is filling: 25 of 100 elements
mountStatus DDS read queue is full (100 elements); data may be lost
mount_Nasmyth_Encoders DDS read queue is filling: 25 of 100 elements
guidingAndOffsets DDS read queue is full (100 elements); data may be lost
currentTargetStatus DDS read queue is full (100 elements); data may be lost
mount_AzEl_Encoders DDS read queue is filling: 25 of 100 elements
mount_AzEl_Encoders python read queue is filling: 24 of 100 elements
measuredTorque DDS read queue is filling: 26 of 100 elements
measuredMotorVelocity DDS read queue is filling: 26 of 100 elements
azEl_mountMotorEncoders DDS read queue is filling: 26 of 100 elements


In [5]:
# slew to target to start focusing - MOUNT DID NOT send inPosition event
await atcs.slew_object('HIP 21837')

Starting new HTTP connection (1): simbad.u-strasbg.fr:80
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
Slewing to HIP 21837: 04 41 42.0316 -12 28 32.813
Auto sky angle: 0.0 deg
Sending command
Stop tracking.
target python read queue is filling: 54 of 100 elements
Unknown tracking state: 9.
Unknown tracking state: 10.
In Position: True.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = +007.869 deg; delta Az = -011.369 deg; delta N1 = -000.000 deg; delta N2 = +000.460 deg [Dome] delta Az = -015.311 deg
[Telescope] delta Alt = +005.858 deg; delta Az = -010.223 deg; delta N1 = +000.000 deg; delta N2 = -000.011 deg [Dome] delta Az = -013.201 deg
[Telescope] delta Alt = +000.669 deg; delta Az = -005.098 deg; delta N1 = -000.000 deg; del

CancelledError: 

In [9]:
# turn off ataos correction so we control 
await atcs.rem.ataos.cmd_disableCorrection.set_start(hexapod=True)

In [10]:
# slew to new target to start focusing
await atcs.slew_object('HIP 27204')

Resetting dropped connection: simbad.u-strasbg.fr
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
Slewing to HIP 27204: 05 45 59.8953 -32 18 23.164
Auto sky angle: 0.0 deg
Sending command
Stop tracking.
target python read queue is filling: 28 of 100 elements
Unknown tracking state: 9.
Unknown tracking state: 10.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -001.746 deg; delta Az = +075.166 deg; delta N1 = -000.000 deg; delta N2 = -063.080 deg [Dome] delta Az = +070.311 deg
[Telescope] delta Alt = -000.482 deg; delta Az = +072.945 deg; delta N1 = -000.000 deg; delta N2 = -057.537 deg [Dome] delta Az = +068.131 deg
[Telescope] delta Alt = +000.001 deg; delta Az = +066.949 deg; delta N1 = -000.000 deg; delta N2 = -053.545 deg [Dome

In [14]:
# take image
await latiss.setup_atspec(filter='empty_1', grating='empty_1')

In [15]:
await latiss.take_object(1, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900166])

In [16]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611104081.6989987, private_rcvStamp: 1611104081.6994717, private_seqNum: 73, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: 1.5740000009536743, userApplied: 1.600000023841858, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [17]:
# leftover offset from failed cwfs - fails because athexapod correction was off
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1240431907, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: hexapod correction must be enabled to reset the offset.')

In [18]:
# turn off ataos correction so we control 
await atcs.rem.ataos.cmd_disableCorrection.set_start(atspectrograph=True)

In [19]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(hexapod=True)

In [20]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611104081.6989987, private_rcvStamp: 1611104081.6994717, private_seqNum: 73, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: 1.5740000009536743, userApplied: 1.600000023841858, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [21]:
# leftover offset from failed cwfs -
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [22]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611104706.9502838, private_rcvStamp: 1611104706.95083, private_seqNum: 74, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: 0.0, userApplied: 0.0, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [23]:
await latiss.take_object(1, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900167])

In [24]:
# Saturated so moving to HIP 26739
await atcs.slew_object('HIP 26739')

Resetting dropped connection: simbad.u-strasbg.fr
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
Slewing to HIP 26739: 05 40 54.4326 -31 59 46.800
Auto sky angle: 0.0 deg
Sending command
Stop tracking.
target python read queue is filling: 36 of 100 elements
Unknown tracking state: 9.
Unknown tracking state: 10.
In Position: True.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = +001.104 deg; delta Az = -000.990 deg; delta N1 = -000.000 deg; delta N2 = -000.286 deg [Dome] delta Az = -002.879 deg
ATDome in position.
Got True
Waiting for telescope to settle.
[Telescope] delta Alt = +000.020 deg; delta Az = -000.034 deg; delta N1 = -000.000 deg; delta N2 = +000.009 deg [Dome] delta Az = -002.879 deg
Telescope in position.


In [25]:
await latiss.take_object(1, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900168])

target DDS read queue is filling: 45 of 100 elements
target python read queue is filling: 44 of 100 elements
target python read queue is filling: 46 of 100 elements


In [26]:
# switching to running the focus script
# check that hexapod went back
await latiss.take_object(1, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900171])

In [27]:
# switching to running the focus script
# check that hexapod went back
await latiss.take_object(60, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900172])

In [28]:
# switching to running the focus script
# check that hexapod went back
await latiss.take_object(300, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900173])

In [ ]:
# check version of pointing component - appears to be incorrect - focus scipt also broken
# going to try to 

In [29]:
# pointing/mount hit limit
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [30]:
# Need to manually say which rotator we're using at the moment (N2-->focus=3, N1-->focus=2)
await atcs.rem.atptg.cmd_focusName.set_start(focus=3)

In [35]:
# Saturated so moving to HIP 26739
# rot_type=2 is parallactic
await atcs.slew_object('HIP 26739', rot=0.0, rot_type=2)

Slewing to HIP 26739: 05 40 54.4326 -31 59 46.800
Setting rotator position with respect to parallactic angle to 0.0 deg.
Parallactic angle: -76.73857723269185 | Sky Angle: 13.261422767308133
Sending command
Stop tracking.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = +003.996 deg; delta Az = +004.281 deg; delta N1 = -000.000 deg; delta N2 = +002.514 deg [Dome] delta Az = -002.879 deg
ATDome in position.
[Telescope] delta Alt = +001.412 deg; delta Az = +001.641 deg; delta N1 = -000.000 deg; delta N2 = +002.394 deg [Dome] delta Az = -002.879 deg
[Telescope] delta Alt = -000.133 deg; delta Az = +000.002 deg; delta N1 = -000.000 deg; delta N2 = +002.122 deg [Dome] delta Az = -002.879 deg
[Telescope] delta Alt = +000.714 deg; delta Az = +000.002 deg; delta N1 

RuntimeError: atptg state is <State.FAULT: 3>, expected <State.ENABLED: 2>

In [36]:
# pointing/mount hit limit
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)

[<State.FAULT: 3>, <State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [37]:
# Need to manually say which rotator we're using at the moment (N2-->focus=3, N1-->focus=2)
await atcs.slew_object('HIP 26739', rot=0.0, rot_type=3)

Slewing to HIP 26739: 05 40 54.4326 -31 59 46.800
Setting rotator physical position to 0.0 deg. Rotator will track sky.
Parallactic angle: -72.44317688139718 | Sky Angle: 23.071401087577726
Sending command
Stop tracking.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = +000.645 deg; delta Az = +001.806 deg; delta N1 = +000.800 deg; delta N2 = +174.509 deg [Dome] delta Az = -002.879 deg
ATDome in position.
[Telescope] delta Alt = +000.002 deg; delta Az = +000.202 deg; delta N1 = +000.000 deg; delta N2 = +173.513 deg [Dome] delta Az = -002.879 deg
[Telescope] delta Alt = +000.001 deg; delta Az = +000.002 deg; delta N1 = +000.007 deg; delta N2 = +172.008 deg [Dome] delta Az = -002.879 deg
[Telescope] delta Alt = +000.000 deg; delta Az = +000.001 deg; delta N1 =

CancelledError: 

In [39]:
await atcs.stop_tracking()

Stop tracking.
Unknown tracking state: 9.
Unknown tracking state: 10.


In [40]:
# going to reload the atptg
# pointing/mount hit limit
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.OFFLINE)

RuntimeError: Error on cmd=cmd_exitControl, initial_state=2: msg='Timed out waiting for command acknowledgement', ackcmd=(ackcmd private_seqNum=565310865, ack=<SalRetCode.OK: 0>, error=0, result='')

In [54]:
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.STANDBY)

[<State.FAULT: 3>, <State.STANDBY: 5>]

In [55]:
# going to reload the atptg
# pointing/mount hit limit
#await asyncio.sleep(1)
await salobj.set_summary_state(atcs.rem.atptg, salobj.State.ENABLED)

[<State.STANDBY: 5>, <State.DISABLED: 1>, <State.ENABLED: 2>]

In [58]:
await atcs.stop_tracking()

Stop tracking.
Unknown tracking state: 9.
Unknown tracking state: 10.


In [59]:
# No longer need to specify rotator
await atcs.slew_object('HD 61597', rot=0.0, rot_type=3)

Resetting dropped connection: simbad.u-strasbg.fr
http://simbad.u-strasbg.fr:80 "POST /simbad/sim-script HTTP/1.1" 200 None
Slewing to HD 61597: 07 37 32.1273 -51 25 16.920
Setting rotator physical position to 0.0 deg. Rotator will track sky.
Parallactic angle: -54.6698742512308 | Sky Angle: 64.8617924836916
Sending command
Stop tracking.
Scheduling check coroutines
process as completed...
atmcs: <State.ENABLED: 2>
atptg: <State.ENABLED: 2>
ataos: <State.ENABLED: 2>
atpneumatics: <State.ENABLED: 2>
athexapod: <State.ENABLED: 2>
atdome: <State.ENABLED: 2>
atdometrajectory: <State.ENABLED: 2>
[Telescope] delta Alt = -019.249 deg; delta Az = -008.510 deg; delta N1 = +000.000 deg; delta N2 = +003.213 deg [Dome] delta Az = -003.038 deg
ATDome in position.
[Telescope] delta Alt = -019.096 deg; delta Az = -007.408 deg; delta N1 = -000.000 deg; delta N2 = +000.097 deg [Dome] delta Az = -003.038 deg
[Telescope] delta Alt = -016.898 deg; delta Az = -002.024 deg; delta N1 = -000.000 deg; delta N2

In [60]:
await latiss.take_object(3, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900174])

In [61]:
await latiss.take_object(300, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900175])

In [65]:
# offset the telescope 20 arcsec in Y 
await atcs.offset_xy(y=-40, x=0, persistent=False)

Calculating x/y offset: 0/-40 
Applying Az/El offset: 36.603765903561914/-16.12961009067604 
Waiting for telescope to settle.
Done


In [66]:
await latiss.take_object(3, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900179])

logMessage DDS read queue is filling: 24 of 100 elements


In [67]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611110484.0036242, private_rcvStamp: 1611110484.0038908, private_seqNum: 80, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: 0.0, userApplied: 0.0, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [78]:
# add focus offset
await atcs.rem.ataos.cmd_setCorrectionModelOffsets.set_start(axis='z', offset=-0.05126616)

In [79]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611111492.6306345, private_rcvStamp: 1611111492.6310325, private_seqNum: 87, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: -0.05126615986227989, userApplied: -0.05126615986227989, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [80]:
await latiss.take_object(3, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001
logMessage DDS read queue is filling: 12 of 100 elements


array([2021011900185])

logMessage DDS read queue is filling: 11 of 100 elements
target DDS read queue is filling: 27 of 100 elements
target python read queue is filling: 26 of 100 elements
target DDS read queue is filling: 54 of 100 elements
target python read queue is filling: 53 of 100 elements


In [81]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611114293.4024913, private_rcvStamp: 1611114293.4070842, private_seqNum: 95, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: -0.05126615986227989, userApplied: -0.05126615986227989, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0
target DDS read queue is full (100 elements); data may be lost
target python read queue is filling: 99 of 100 elements
logMessage DDS read queue is filling: 17 of 100 elements


In [82]:
# reset the offsets
await atcs.rem.ataos.cmd_resetOffset.set_start(axis='z')

In [83]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611115585.4930444, private_rcvStamp: 1611115585.4935281, private_seqNum: 99, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: 0.0, userApplied: 0.0, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [84]:
# now apply previous offset
await atcs.rem.ataos.cmd_offset.set_start(z=-0.05126615)

In [85]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611115671.2679672, private_rcvStamp: 1611115671.2684307, private_seqNum: 100, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: -0.051266148686409, userApplied: -0.051266148686409, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [86]:
# now add in the offset that was just calculated
await atcs.rem.ataos.cmd_offset.set_start(z=0.0067265)

In [87]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611115724.9480515, private_rcvStamp: 1611115724.948449, private_seqNum: 101, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: -0.04453964903950691, userApplied: -0.04453964903950691, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [88]:
await latiss.take_object(15, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900203])

logMessage DDS read queue is filling: 12 of 100 elements
target DDS read queue is filling: 89 of 100 elements
target python read queue is filling: 88 of 100 elements
target python read queue is filling: 10 of 100 elements


In [92]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611116516.7070408, private_rcvStamp: 1611116516.707486, private_seqNum: 106, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: 0.7554603815078735, userApplied: 0.7554603815078735, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [93]:
# hexapod got stuck in extra focal
await atcs.rem.ataos.cmd_offset.set_start(z=-0.8)

In [94]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611116580.2333057, private_rcvStamp: 1611116580.2337081, private_seqNum: 107, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: -0.04453964903950691, userApplied: -0.04453964903950691, filter: 0.0, disperser: 0.0, wavelength: 0.0, priority: 0


In [95]:
await latiss.take_object(20, filter='empty_1', grating='empty_1')

Generating group_id
OBJECT 0001 - 0001


array([2021011900208])

target DDS read queue is filling: 28 of 100 elements
target python read queue is filling: 27 of 100 elements
target DDS read queue is filling: 66 of 100 elements
target python read queue is filling: 65 of 100 elements
target python read queue is filling: 53 of 100 elements


In [96]:
tmp = atcs.rem.ataos.evt_correctionEnabled.get()
print(tmp)

private_revCode: 55f0662b, private_sndStamp: 1611119577.3618457, private_rcvStamp: 1611119577.3624682, private_seqNum: 20, private_identity: ATAOS, private_origin: 165593, private_host: 0, m1: True, m2: False, hexapod: True, focus: False, atspectrograph: False, moveWhileExposing: False, priority: 0
logMessage DDS read queue is filling: 53 of 100 elements
logMessage DDS read queue is filling: 14 of 100 elements


In [97]:
# check what the focus offsets are as we don't know why the focus is so bad
tmp=await atcs.rem.ataos.evt_focusOffsetSummary.aget()
print(tmp)

private_revCode: d8296953, private_sndStamp: 1611125616.692829, private_rcvStamp: 1611125616.693099, private_seqNum: 184, private_identity: ATAOS, private_origin: 165593, private_host: 0, total: -1.7058463096618652, userApplied: -0.06784626096487045, filter: 0.0, disperser: -0.026000000536441803, wavelength: 0.0, priority: 0
